### Utterances por frase sin simbolos


In [10]:
#import pylangacq as pla
import pandas as pd
import spacy
import glob
import re
import spacy
from sklearn.utils import shuffle

In [2]:
eve = pla.read_chat('./Corpus/Pitt/Control/cookie/*.cha', encoding = 'utf-8')
tagged = eve.tagged_sents(participant='PAR')
sentences = eve.sents(participant='PAR')
group = list(['control']*len(tagged))
control_df = pd.DataFrame(
    {'utterances': sentences,
     'tagged': tagged,
     'group': group
    })

In [3]:
eve = pla.read_chat('./Corpus/Pitt/Dementia/cookie/*.cha', encoding = 'utf-8')
tagged = eve.tagged_sents(participant='PAR')
sentences = eve.sents(participant='PAR')
group = list(['dementia']*len(tagged))
dementia_df = pd.DataFrame(
    {'utterances': sentences,
     'tagged': tagged,
     'group': group
    })

In [7]:
# Unimos los dos df , reorganizamos random y pasamos la colummna utterances de list -> str
from sklearn.utils import shuffle

df = pd.concat([control_df, dementia_df])
df = shuffle(df).reset_index(drop=True)

utterances_str = []
for i in range(len(df['utterances'])):
    list1 = df['utterances'][i]
    str1 = ' '.join(list1)
    utterances_str.append(str1)
    
df['utterances_str'] = utterances_str
df.head()

,utterances,tagged,group,utterances_str
0,"[that's, about, all, .]","[(that's, PRO:DEM, that, (1, 2, SUBJ)), (CLITI...",dementia,that's about all .
1,"[what's, it, always, say, that, ?]","[(what's, PRO:INT, what, (1, 5, SUBJ)), (CLITI...",dementia,what's it always say that ?
2,"[I, don't, see, anything, else, happening, .]","[(I, PRO:SUB, I, (1, 4, SUBJ)), (don't, MOD, d...",control,I don't see anything else happening .
3,"[and, the, mother, is, washing, the, dishes, b...","[(and, COORD, and, (1, 5, LINK)), (the, DET:AR...",control,and the mother is washing the dishes but she's...
4,"[a, boy, is, getting, a, cookie, out_of, a, co...","[(a, DET:ART, a, (1, 2, DET)), (boy, N, boy, (...",control,a boy is getting a cookie out_of a cookie jar ...


In [6]:
data = df[['group', 'utterances_str']].rename(columns={"group":"label", "utterances_str":"text"})
data.head()

,label,text
0,control,is that it ?
1,dementia,and he's getting cookies .
2,control,the water's coming out_of the faucet .
3,dementia,she's doing the dishes .
4,dementia,and the woman is drying dishes .


In [9]:
data['label'].value_counts()

dementia    3908
control     3145
Name: label, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

df = data.copy()

X = df['text']  # this time we want to look at the text
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(4725, 1403)

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(4725, 1403)

In [14]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train) 

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [18]:
# Form a prediction set
predictions = text_clf.predict(X_test)
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[615 445]
 [346 922]]


In [19]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

     control       0.64      0.58      0.61      1060
    dementia       0.67      0.73      0.70      1268

   micro avg       0.66      0.66      0.66      2328
   macro avg       0.66      0.65      0.65      2328
weighted avg       0.66      0.66      0.66      2328



In [20]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.6602233676975945


### Utterances por dialogo con simbolos con spacy


In [3]:
nlp = spacy.load('en_core_web_sm')

In [5]:
path_cha = './Corpus/Pitt/Control/cookie/*.cha'
#path_cha = './Corpus/Pitt/Dementia/cookie/*.cha'

processed = []
def process_with_symbols(path_cha):
    import glob
    import re
    
    files = sorted(glob.glob(path_cha))
    
    processed = []
    
    for file in files:
        document = open(file).read()
        doc = document.replace('\t', ' ').split('\n')
        
        
        pre_list = []
        for d in doc:
            match = re.findall(r'\*PAR:(.*)',d)
            if (len(match) != 0):
                pre_list.append(match)

        listt = []
        for p in pre_list:
            match = re.sub('x15.*?x15', '', str(p))
            match = re.sub('[\\\]', '', match)
            match = re.sub('["""]', '', match)
            match = re.sub('[\\\']', '', match)
            match = re.search(r'.*?\[(.*)].*', match).group(1)
            listt.append(match)
        
        processed.append(listt)
    
    return(processed)
    

utterances_symbol = process_with_symbols(path_cha) 
utterances_symbol_control = utterances_symbol
len(utterances_symbol_control)
    

243

In [6]:
#path_cha = './Corpus/Pitt/Control/cookie/*.cha'
path_cha = './Corpus/Pitt/Dementia/cookie/*.cha'

processed = []
def process_with_symbols(path_cha):
    import glob
    import re
    
    files = sorted(glob.glob(path_cha))
    
    processed = []
    
    for file in files:
        document = open(file).read()
        doc = document.replace('\t', ' ').split('\n')
        
        
        pre_list = []
        for d in doc:
            match = re.findall(r'\*PAR:(.*)',d)
            if (len(match) != 0):
                pre_list.append(match)

        listt = []
        for p in pre_list:
            match = re.sub('x15.*?x15', '', str(p))
            match = re.sub('[\\\]', '', match)
            match = re.sub('["""]', '', match)
            match = re.sub('[\\\']', '', match)
            match = re.search(r'.*?\[(.*)].*', match).group(1)
            listt.append(match)
        
        processed.append(listt)
    
    return(processed)
    

utterances_symbol = process_with_symbols(path_cha)    


utterances_symbol_dementia = utterances_symbol
len(utterances_symbol_dementia)

309

In [21]:
group = list([0]*len(utterances_symbol_control)) # Control = 0
control_df = pd.DataFrame(
    {'label': group,
     'text': utterances_symbol_control
     })

In [22]:
group = list([1]*len(utterances_symbol_dementia)) # Dementia = 1
dementia_df = pd.DataFrame(
    {'label': group,
     'text': utterances_symbol_dementia
     })

In [23]:
# Unimos los dos df , reorganizamos random y pasamos la colummna utterances de list -> str
df_spacy = pd.concat([control_df, dementia_df])
df_spacy = shuffle(df_spacy).reset_index(drop=True)
df_spacy.head(20)

,label,text
0,1,[ well the [/] &uh the little boy is (.) steal...
1,0,[ okay mother is drying the dishes but the wat...
2,1,"[ (.) what ? [+ exc] , the boys &uh handing &..."
3,0,[ the boy is &uh tipping stool reaching for th...
4,0,[ well the kids &uh stealin(g) cookies out o(f...
5,0,"[ things that are happening . [+ exc] , girls..."
6,1,[ this is &uh a clause copy [/] copy c@l by hi...
7,1,[ well theres a little boy <on the> [//] on a ...
8,1,"[ the sink is running over . , and &uh the [/..."
9,1,"[ now wait a minute . [+ exc] , everything th..."


In [24]:
df = df_spacy.copy()
text = df['text']

to_df = []


for i in text:
    to_df.append(' '.join(i))

df['text'] = to_df

In [25]:
to_df[0]

' well the [/] &uh the little boy is (.) stealing cookies and  hes &uh partially gonna fall off the stool .   his sister is tellin(g) him to be quiet .   the mother is washing dishes .   an(d) &uh the water is overflowing onto the floor and shes'

In [28]:
from sklearn.model_selection import train_test_split 

X = df['text']  # this time we want to look at the text
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(441, 1702)

In [33]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(441, 1702)

In [34]:
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
                    ])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [35]:
# Form a prediction set
predictions = text_clf.predict(X_test)
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[38 12]
 [16 45]]


In [36]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.70      0.76      0.73        50
           1       0.79      0.74      0.76        61

   micro avg       0.75      0.75      0.75       111
   macro avg       0.75      0.75      0.75       111
weighted avg       0.75      0.75      0.75       111



In [73]:
#clf = LinearSVC()
#clf.fit(X_train_tfidf,y_train)

#text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                    # ('clf', LinearSVC()),
                   # ])

vectorizer = CountVectorizer()
               
        
# Feed the training data through the pipeline
#text_clf.fit(X_train, y_train)



#LinearSVC() es el clasificador

pipeline = Pipeline(steps=[('vect', vectorizer), # Imposible poner más de 1 núcleo con CountVectorizer. Pra optimizar los hiperparámetros
                           ('cls', LinearSVC()), # Primero he tendio que optimizar el modelo por un lado y por otro el vectorizador. Si no, era imposible.
                    ])
 
#pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                   #  ('clf', LinearSVC()),
                   # ])

#Aqui definimos el espacio de parámetros a explorar
parameters = {'vect__max_df': [1.9],
              'vect__max_features': [1000],
              'vect__min_df': [50],
              'vect__ngram_range': [(1, 1)],
              'cls__C': (0.001, 0.0015, 0.01, 1, 10, 100),
              'cls__loss': ['squared_hinge'],
              'cls__max_iter': [1000],
            }



grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=parameters,
                           scoring='roc_auc',
                           verbose=3,
                           n_jobs = -1,
                           cv=20                          
                          )
                         


grid_search.fit(X_train, y_train)


Fitting 20 folds for each of 6 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    2.2s finished
/home/gsus/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=20, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': [1.9], 'vect__max_features': [1000], 'vect__min_df': [50], 'vect__ngram_range': [(1, 1)], 'cls__C': (0.001, 0.0015, 0.01, 1, 10, 100), 'cls__loss': ['squared_hinge'], 'cls__max_iter': [1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=3)

In [74]:
grid_search.best_params_

{'cls__C': 0.0015,
 'cls__loss': 'squared_hinge',
 'cls__max_iter': 1000,
 'vect__max_df': 1.9,
 'vect__max_features': 1000,
 'vect__min_df': 50,
 'vect__ngram_range': (1, 1)}

In [75]:
best_grid_search = grid_search.best_estimator_
best_grid_search

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.9, max_features=1000, min_df=50,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [76]:
# Form a prediction set
predictions = best_grid_search.predict(X_test)
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[35 15]
 [ 9 52]]


In [77]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.80      0.70      0.74        50
           1       0.78      0.85      0.81        61

   micro avg       0.78      0.78      0.78       111
   macro avg       0.79      0.78      0.78       111
weighted avg       0.78      0.78      0.78       111



In [79]:
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)


pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
                    ])

parameters = {
              'clf__C': (0.001, 0.0015, 0.01, 1, 10, 100),
              'clf__loss': ['squared_hinge'],
              'clf__max_iter': [1000],
            }



grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=parameters,
                           scoring='roc_auc',
                           verbose=3,
                           n_jobs = -1,
                           cv=20                          
                          )

                       


grid_search.fit(X_train, y_train)

Fitting 20 folds for each of 6 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    3.3s finished
/home/gsus/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=20, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__C': (0.001, 0.0015, 0.01, 1, 10, 100), 'clf__loss': ['squared_hinge'], 'clf__max_iter': [1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=3)

In [80]:
grid_search.best_params_

{'clf__C': 1, 'clf__loss': 'squared_hinge', 'clf__max_iter': 1000}

In [81]:
best_grid_search = grid_search.best_estimator_
best_grid_search

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [82]:
# Form a prediction set
predictions = best_grid_search.predict(X_test)
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[38 12]
 [16 45]]


In [83]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.70      0.76      0.73        50
           1       0.79      0.74      0.76        61

   micro avg       0.75      0.75      0.75       111
   macro avg       0.75      0.75      0.75       111
weighted avg       0.75      0.75      0.75       111

